<a href="https://colab.research.google.com/github/austinAbraham/hello-world/blob/master/pyspark_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mango DE PySpark Test

This exercise is designed to test a candidates ability to translate requirements into a very small ETL script using PySpark.

The first set of cells in this notebook install java and spark in the environment, a repo with a data generating function, and start a spark session. One common issue with using these setup instructions is slow response from the mirror used to download spark, so an alternative mirror is provided (commented out in code cell). However, you may need to shop around to find a mirror that is more appropriate for your location. Candidates are expected to be able to debug installation/setup issues themselves.

Instructions for the test can be found below the setup cells.

If you are new to Google colab, note that there is a directory browser button on the left. As you run through the setup cells and generate data for the test, files will be added here. The home directory for this workspace is /content and you can mount this to a personal Drive to persist files that are added here if you wish. For more information on colab, see the tutorial here: https://colab.research.google.com/notebooks/welcome.ipynb


In [48]:
# Install Java and Spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

!wget -qN https://apache.mirrors.nublue.co.uk/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz
#!wget -qN https://mirrors.gethosted.online/apache/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz

!tar xf spark-3.0.3-bin-hadoop2.7.tgz

In [49]:
# If having issues then uncomment the below line and run. Then re-run the previous two cells
# !rm -rf spark-2.4.7-bin-hadoop2.7*

In [50]:
!git clone https://github.com/MangoTheCat/de_tech_test_pyspark.git functions > /dev/null 2>&1

In [51]:
!pip install -q findspark

In [52]:
# Configure Java / Spark environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"

In [53]:
# start a spark session to check that this is all working
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

## Scenario overview

An organisation recieves files, usually daily but with some gaps, which are saved to ~/data/main. The task is to do some basic cleaning and transformation of these files and to write them to a separate dataset. Imagine that the output you produce here will be registered as an external table in a Hive database, which an analysis team will query using a SQL tool.

Files are generated in the first code cell below. Running this cell will produce a new file with random data for today's date. It will throw an error if a file already exists for today's date. Each file will be relatively small due to the restrictions of this environment, but treat these files as though they are large.

Detailed instructions of the required transformations are expressed below. Feel free to use cells in this notebook while developing your solution, but you should develop a solution that can sit in a separate .py file. You should also include unit tests as appropriate. Remember that, if you have not mounted the drive of your colab runtime, any external files you create here will not persist. Please add a text cell describing any assumptions you have made about how your script should be run.

In [69]:
from functions.generate_data import generate_data
generate_data()


## Requirements

1. Some values in field1 are missing. We know the value columns for these records are faulty so we should remove these rows entirely.

2. Field2 also has some missing data, but we can work out what those values should be. If field3 is between 8 and 12, these missing values should be equal to 1. Otherwise, they should be 2.

3. We need to add an extra dimension based on fields 1 and 2. A lookup csv has been provided in the data directory. You should add the lookup_val field to the dataset as a column named 'new_dimension'.

4. If either field1 or field2 have a value of 10, the new_dimension column should have a value of 'XX'.

5. The analysis team will often run queries over a specific year or a specific year/month. Take this into consideration when designing the output table structure.

6. For analysis purposes, the team won't need field3. Please remove this field and aggregate on the remaining fields. All the value columns should be summed.

7. A distinct count of the values in field3 should be included in the output.

8. The analysts would also like a column added showing the proportion of val3 for each value of field4 over the total of val3 for the remaining dimension columns.

9. It should be rare, but we may sometimes receive a replacement file for an old period (which will overwrite the old file in the data/main directory). We need to be able to use the transformation code on an adhoc basis to update that data in our output. The cluster we are working on is very busy, so we don't want this to take longer than it has to.

10. Write your output to the home directory in a format and structure that you deem suitable.

In [70]:
import pandas as pd
import numpy as np
from pyspark.sql.functions import expr, when, col

In [71]:
data = spark.read.csv('./data/main/2021-07-23/test_data.csv', header=True)
data_lookup = spark.read.csv('./data/lookup.csv', header=True)
print(data.printSchema())
print(data.count())
data.show(n=5)

# print(data.describe())
data_work_copy = data

root
 |-- field1: string (nullable = true)
 |-- field2: string (nullable = true)
 |-- field3: string (nullable = true)
 |-- field4: string (nullable = true)
 |-- val1: string (nullable = true)
 |-- val2: string (nullable = true)
 |-- val3: string (nullable = true)

None
1000000
+------+------+------+------+------------------+------------------+------------------+
|field1|field2|field3|field4|              val1|              val2|              val3|
+------+------+------+------+------------------+------------------+------------------+
|    18|     2|    11|    19| 1103.739025081271|3998.2667294575836|10388.250709799426|
|    15|     1|     4|     1| 974.8860468086983| 8220.326105064129| 428.1437099985022|
|    17|     1|    14|     7|1107.3628010812563|14818.451818177093| 3260.553371172561|
|    18|     2|     2|     8|1166.3007794734226|14013.300988118377| 5627.275134246259|
|    19|     9|    14|     9|1084.5596242024433| 792.5937993220928| 7107.804877175669|
+------+------+------+---

In [72]:
data_work_copy = data.dropna(how='any', subset='field1')
data_work_copy.count()

999995

In [86]:
data_work_copy.select('field2').distinct().show()
data_work_copy.printSchema()
df1 = data_work_copy\
  .withColumn('field2_copy', when(col('field2')== 'null', \
                                  when(col('field3') < 12, when(col('field3') > 8, 1).otherwise(2)).otherwise(2)))\
  .drop('field2')\
  .withColumnRenamed('field2_copy', 'field2')

df1.printSchema()
df1.show()

+------+
|field2|
+------+
|     7|
|    15|
|    11|
|     3|
|     8|
|    16|
|  null|
|     5|
|    18|
|    17|
|     6|
|    19|
|     9|
|     1|
|    10|
|     4|
|    12|
|    13|
|    14|
|     2|
+------+

root
 |-- field1: string (nullable = true)
 |-- field2: string (nullable = true)
 |-- field3: string (nullable = true)
 |-- field4: string (nullable = true)
 |-- val1: string (nullable = true)
 |-- val2: string (nullable = true)
 |-- val3: string (nullable = true)

root
 |-- field1: string (nullable = true)
 |-- field3: string (nullable = true)
 |-- field4: string (nullable = true)
 |-- val1: string (nullable = true)
 |-- val2: string (nullable = true)
 |-- val3: string (nullable = true)
 |-- field2: integer (nullable = true)

+------+------+------+------------------+------------------+------------------+------+
|field1|field3|field4|              val1|              val2|              val3|field2|
+------+------+------+------------------+------------------+----------------

In [148]:
data_lookup = data_lookup.withColumnRenamed('field1', 'f1')
data_lookup.printSchema()
df1.printSchema()
data_joined = df1.join(data_lookup, on=[col('field1')==col('f1'), col('field2')==col('f2')])\
  .withColumnRenamed('lookup_val', 'new_dimension.      ')
data_joined.printSchema()
data_joined.show()

root
 |-- f1: string (nullable = true)
 |-- f2: string (nullable = true)
 |-- lookup_val: string (nullable = true)

root
 |-- field1: string (nullable = true)
 |-- field3: string (nullable = true)
 |-- field4: string (nullable = true)
 |-- val1: string (nullable = true)
 |-- val2: string (nullable = true)
 |-- val3: string (nullable = true)
 |-- field2: string (nullable = true)

root
 |-- field1: string (nullable = true)
 |-- field3: string (nullable = true)
 |-- field4: string (nullable = true)
 |-- val1: string (nullable = true)
 |-- val2: string (nullable = true)
 |-- val3: string (nullable = true)
 |-- field2: string (nullable = true)
 |-- f1: string (nullable = true)
 |-- f2: string (nullable = true)
 |-- new_dimension: string (nullable = true)

+------+------+------+------------------+------------------+------------------+------+---+---+-------------+
|field1|field3|field4|              val1|              val2|              val3|field2| f1| f2|new_dimension|
+------+------+------

In [147]:
df2 = data_joined.withColumn('lookup_val_New', when(((col('field1') == 10) | (col('field2') == 10)), 'XX').otherwise(col('lookup_val')))\
  .drop('lookup_val')\
  .withColumnRenamed('lookup_val_New', 'lookup_val')

df2.select('lookup_val').distinct().show(n=50) 

data_joined.select('field2').distinct().show()

+----------+
|lookup_val|
+----------+
|         K|
|         l|
|         x|
|         g|
|         F|
|         Q|
|         m|
|         E|
|         T|
|         f|
|         B|
|         n|
|         k|
|         Y|
|         M|
|         L|
|         V|
|         U|
|         v|
|         e|
|         O|
|         D|
|         o|
|         h|
|         z|
|         C|
|         p|
|         J|
|         d|
|         Z|
|         A|
|         N|
|         X|
|         y|
|         w|
|         c|
|         W|
|         S|
|         u|
|         R|
|         i|
|         G|
|         q|
|         j|
|         b|
|         I|
|         P|
|         a|
|         r|
|         t|
+----------+
only showing top 50 rows

+------+
|field2|
+------+
|     7|
|    15|
|    11|
|     3|
|     8|
|    16|
|     5|
|    18|
|    17|
|     6|
|    19|
|     9|
|     1|
|     4|
|    12|
|    13|
|    14|
|     2|
+------+



In [77]:
data_l1 = data_lookup.select( 'f2').distinct()
data_l1.show()


df1_ct = df1.select('field2').distinct()
df1_ct.show()

+---+
| f2|
+---+
|  7|
| 15|
| 11|
|  3|
|  8|
| 16|
|  5|
| 18|
| 17|
|  6|
| 19|
|  9|
|  1|
|  4|
| 12|
| 13|
| 14|
|  2|
+---+

+------+
|field2|
+------+
|  null|
+------+

